In [1]:
import torch
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset, random_split
# Summary-like from Tensorflow
# from torchinfo import summary
# Import Numpy
import numpy as np
# Plot Import
import matplotlib.pyplot as plt
# For timer
import time
# Import OS
import os
from statistics import mean
# WFDB
import wfdb
# ==== LOCAL LIBS ====
import ecg_tools_lite as et
import evaluator as eva
import trainer as tr
import models as mod
# math
import math

from torchsummary import summary

In [2]:
# What are you trying to do?
create_new_model = True
create_new_res = True
# If create_new_model is True, epochs must have a value
epochs = 5
# If create_new_model is False, load_model_name must have a value
load_model_name = 'saved_model_dict.pt'
ecg_saved_res = 'res_pt_full.npy'
# Normalization option (either option 0 or 1)
norm_option = 2

In [4]:
ecg_noisy = torch.from_numpy(np.load('all_none_fs1024_e06.npy')).double()
ecg_clean = torch.from_numpy(np.load('all_none_fs1024_e24.npy')).double()

# Acquire original shape
x = ecg_noisy.shape[0]
y = ecg_noisy.shape[2]
z = ecg_noisy.shape[1]

# Reshape
ecg_noisy = torch.reshape( ecg_noisy, (x, y, z))
ecg_clean = torch.reshape( ecg_clean, (x, y, z))

print(f'New shape: {ecg_noisy.shape}')

# == Basic Normalization technique (norm from 0 to 1)
# ecg_noisy = et.norm_basic( ecg_noisy )
# ecg_clean = et.norm_basic( ecg_clean )

# == Normalization technique (Norm from -1 to 1)
ecg_noisy = et.norm_sig( ecg_noisy )
ecg_clean = et.norm_sig( ecg_clean )

# ecg_noisy = ecg_noisy.cpu().numpy().flatten()
# ecg_clean = ecg_clean.cpu().numpy().flatten()

# ecg_noisy = et.realign_all_chunks(ecg_noisy, ecg_clean)

New shape: torch.Size([5544, 1, 1024])


In [5]:
kernel_size=kernel_size = 19
pad_size = 9
class cnn_denoiser(nn.Module):    
    def __init__(self):
        super(cnn_denoiser, self).__init__()

        self.denoiser = nn.Sequential(
            nn.Conv1d(1, 36, kernel_size=kernel_size, stride=1, padding=pad_size),
            nn.BatchNorm1d(36),
            nn.ReLU(True),
            # nn.AvgPool1d(kernel_size=2, stride=4),

            nn.Conv1d(36, 36, kernel_size=kernel_size, stride=1, padding=pad_size),
            nn.BatchNorm1d(36),
            nn.ReLU(True),
            # nn.AvgPool1d(kernel_size=2, stride=4),

            nn.Conv1d(36, 36, kernel_size=kernel_size, stride=1, padding=pad_size),
            nn.BatchNorm1d(36),
            nn.ReLU(True),
            # nn.AvgPool1d(kernel_size=2, stride=4),

            nn.Conv1d(36, 36, kernel_size=kernel_size, stride=1, padding=pad_size),
            nn.BatchNorm1d(36),
            nn.ReLU(True),
            # nn.AvgPool1d(kernel_size=2, stride=4),

            nn.Conv1d(36, 36, kernel_size=kernel_size, stride=1, padding=pad_size),
            nn.BatchNorm1d(36),
            nn.ReLU(True),
            # nn.AvgPool1d(kernel_size=2, stride=4),

            nn.Conv1d(36, 36, kernel_size=kernel_size, stride=1, padding=pad_size),
            nn.BatchNorm1d(36),
            nn.ReLU(True),
            # nn.AvgPool1d(kernel_size=2, stride=4),

            # nn.Linear(36, 36)
        )

    def forward(self, x):
        x = self.denoiser(x)
        return x

In [6]:
# test_data = torch.tensor([[2,2,2,2]])# np.array([2,2,2])
# test_data = torch.from_numpy(test_data)
model = cnn_denoiser()
model.double()

test_data = torch.ones([1, 1, 1024], dtype=torch.double)

print( test_data.shape )

# train_res = model( test_data )

losses = tr.train_model( model=model, epochs=2, ecg_noisy=ecg_noisy, ecg_clean=ecg_clean, train_pct=0.8)

torch.Size([1, 1, 1024])
train_size[4435] + valid_size[1109]= 5544
same size
C:\python\lib\site-packages\torch\nn\modules\loss.py:446: UserWarning: Using a target size (torch.Size([1, 1, 1024])) that is different to the input size (torch.Size([1, 36, 1024])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Epoch 1 of 2 || time: 302.98 || loss = 0.29342382253745103
Epoch 2 of 2 || time: 297.44 || loss = 0.2852555636927421
Elapsed time: 600.42, (in mins: 10:0)
Validation dataset has not been used: Available validex set size = 1109


In [7]:
summary( cnn_denoiser())

Layer (type:depth-idx)                   Param #
├─Sequential: 1-1                        --
|    └─Conv1d: 2-1                       720
|    └─BatchNorm1d: 2-2                  72
|    └─ReLU: 2-3                         --
|    └─Conv1d: 2-4                       24,660
|    └─BatchNorm1d: 2-5                  72
|    └─ReLU: 2-6                         --
|    └─Conv1d: 2-7                       24,660
|    └─BatchNorm1d: 2-8                  72
|    └─ReLU: 2-9                         --
|    └─Conv1d: 2-10                      24,660
|    └─BatchNorm1d: 2-11                 72
|    └─ReLU: 2-12                        --
|    └─Conv1d: 2-13                      24,660
|    └─BatchNorm1d: 2-14                 72
|    └─ReLU: 2-15                        --
|    └─Conv1d: 2-16                      24,660
|    └─BatchNorm1d: 2-17                 72
|    └─ReLU: 2-18                        --
Total params: 124,452
Trainable params: 124,452
Non-trainable params: 0


Layer (type:depth-idx)                   Param #
├─Sequential: 1-1                        --
|    └─Conv1d: 2-1                       720
|    └─BatchNorm1d: 2-2                  72
|    └─ReLU: 2-3                         --
|    └─Conv1d: 2-4                       24,660
|    └─BatchNorm1d: 2-5                  72
|    └─ReLU: 2-6                         --
|    └─Conv1d: 2-7                       24,660
|    └─BatchNorm1d: 2-8                  72
|    └─ReLU: 2-9                         --
|    └─Conv1d: 2-10                      24,660
|    └─BatchNorm1d: 2-11                 72
|    └─ReLU: 2-12                        --
|    └─Conv1d: 2-13                      24,660
|    └─BatchNorm1d: 2-14                 72
|    └─ReLU: 2-15                        --
|    └─Conv1d: 2-16                      24,660
|    └─BatchNorm1d: 2-17                 72
|    └─ReLU: 2-18                        --
Total params: 124,452
Trainable params: 124,452
Non-trainable params: 0

In [8]:
ecg_res = model.denoiser(ecg_noisy)
print( f'ecg_res shape: {ecg_res.shape}')
ec_res = ecg_res.flatten()

RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:73] data. DefaultCPUAllocator: not enough memory: you tried to allocate 31064850432 bytes. Buy new RAM!

In [ ]:
et.ecg_plot([ecg_clean, ecg_res], ['ECG Clean', 'ECG Result'])

In [ ]:
# train_pct = 0.8
# def get_train_valid_sizes( ecg_clean ):
#     # valid_pct = 1 - train_pct # auto compute for the validation
#     # Compute for the sizes of training and validation
#     total_size = ecg_clean.shape[0]
#     train_size = int( (total_size) * train_pct )
#     valid_size = total_size - train_size
    
#     print( f'train_size[{train_size}] + valid_size[{valid_size}]= {total_size}')
#     if( (train_size+valid_size) == total_size):
#         print(f'same size')
#     else:
#         print(f'not same size')
#     return train_size, valid_size, total_size

# # CREATE THE TRAINDEX (TRAINING INDEX SETS)
# def create_index_loaders( ecg_clean ):
#     # Get train and validation set sizes
#     train_size, valid_size, total_size = get_train_valid_sizes(ecg_clean)    
#     # Well instead of having to randomize the data itself, why not the numbers used to index
#     index_set = np.arange(0, total_size)

#     # split the indexes used for training and validation
#     train_indexset, val_indexset = random_split( index_set, (train_size, valid_size))

#     # Create DataLoaders for the corresponding train and val sets
#     traindex_loader = DataLoader( train_indexset, shuffle=True, batch_size=1 )
#     validex_loader = DataLoader( val_indexset, shuffle=True, batch_size=1 )
#     return traindex_loader, validex_loader

In [ ]:
traindex_loader, validex_loader = create_index_loaders( ecg_clean )

In [ ]:
# model = cnn_denoiser().cuda()
# model.double()

# for epoch in range(1):
#     running_loss = 0.0

#     for i, data in enumerate(traindex_loader):
#         index = data.numpy()[0]
#         clean_samp = ecg_clean[index]

#         clean_samp = clean_samp.view(1, clean_samp.shape[0], clean_samp.shape[1])
#         print(clean_samp.shape)
#         train_res = model( clean_samp )